<a href="https://colab.research.google.com/github/james-trayford/AudibleUniverseWorkbooks/blob/group4/STRAUSSdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook prepared by **Dr James Trayford** - for queries please email [`james.trayford@port.ac.uk`](mailto:james.trayford@port.ac.uk)

To use this notebook (if you haven't already) you can first save a copy to your local drive by clicking `File > Save a Copy in Drive` and run on that copy. `Edit > Clear all outputs` on that copy should also ensure yopu have a clean version
 to start from.

## **0. Introduction**




We will be using the [STRAUSS code](https://github.com/james-trayford/strauss) for this activity

<img src="https://github.com/james-trayford/strauss/blob/main/misc/strauss_logo.png?raw=true">

For reference, you can read an overview of the code (as well as detailed documentation) [at this link](https://strauss.readthedocs.io/en/latest/)


`strauss` is an open source, object-oriented python library intended to be a flexible toolkit and engine for sonification, allowing detailed low-level control over the sonification process. Simultaneously, casual users can quickly hear their data, adapting a library of python notebook templates for a range of applications. 

By analogy to visualisation, the intention is to provide something akin to a plotting library. A library allows users to make a variety of simple plots easily, but also the option to control all aspects of plots and adapt them to the intricacies of their data, for optimal presentation. 

By adopting a general approach, `strauss` is intended to sonify any form of data for users with differing expertise. `strauss` is work in progress, and benefits form user feedback - filling in this feedback will be very useful in making the code better!


### **This notebook:** 
This notebook will demonstrate some of the ways in which `strauss` can be applied to sonify light curves. Alternative options may be demonstrated with commented out code ( i.e. lines of actual code preceded by `#`) - feel free to try these! Generally the goal of this notebook is to give some open examples to explore the code and experiment, so please do so! 

### **STRAUSS video**

You can also run the below cell to see a 12 minute introduction video

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('yhSNM8ztSEk', width=800, height=600) 

## **1. Setup:**

First, let's install `strauss`! Just run the code cell below.

*We will use the `spectraliser` development branch for this notebook. Install can take a while - but you should only need to run it once!*


In [ ]:
 !pip --quiet install git+https://github.com/james-trayford/strauss.git@spectraliser -U

and also make a local copy of the repository

In [ ]:
 !git clone https://github.com/james-trayford/strauss.git

Make plots appear in-line by default

In [ ]:
%matplotlib inline

Import the modules we need...

In [ ]:
%load_ext autoreload
%autoreload 1

# strauss imports
from strauss.sonification import Sonification
from strauss.sources import Events, Objects
from strauss import channels
from strauss.score import Score
from strauss.generator import Sampler, Synthesizer, Spectralizer
from strauss import sources as Sources 
import strauss

# other useful modules
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.signal import savgol_filter
import urllib.request
import os
import zipfile
import glob
import yaml
import pandas as pd

# modules to display in-notebook
import IPython.display as ipd
from IPython.core.display import display, Markdown, Latex, Image

# set figures to be a decent size by default
import matplotlib
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 18}
matplotlib.rc('font', **font)
matplotlib.rc('figure', **{'figsize':[14.0, 7.0]})

## **2. Getting the Data**

⚠  ***The cells in this section are to organise the data we need for the session, don't worry too much about the details of the code here!*** ⚠ 

First, let's download the data for this session to somewhere we have `Colab` access

In [ ]:
outdir = "./AU2_Group4"

path = os.path.realpath(outdir)
if not glob.glob(outdir): 
  os.mkdir(path)  
    
fname = "Group4_input_data.zip"
url = "https://drive.google.com/uc?export=download&id=1hZYDckpHnWWdZXOAavFOcieeZP42WGJr"

print(f"Downloading files...")
with urllib.request.urlopen(url) as response, open(f"{path}/{fname}", 'wb') as out_file:
  data = response.read() # a `bytes` object
  out_file.write(data)

print(f"Unzipping files to {outdir}/Data ...")
with zipfile.ZipFile(f"{outdir}/{fname}", 'r') as zip_ref:
    zip_ref.extractall(f"{outdir}")

print(f"Clearing up...")
os.remove(f"{path}/{fname}")

print("Done.")

We now have access to all the data for this group - let's see what's available and make some plots...

In [ ]:
for f in glob.glob('./AU2_Group4/Data/csv/*.csv'):
  data_table = pd.read_csv(f)
  display(Markdown(f"### File: <mark>`{f}`</mark>"))
  xlab = data_table.columns[0]
  ylab = data_table.columns[1]
  plt.plot(data_table[xlab], data_table[ylab])
  plt.xlabel(f'{xlab}')
  plt.ylabel(f'{ylab}')
  plt.show()
  print(data_table.head())
  #display(data_table)
  

## **3. One-dimensional time series sonification**

Here we will sonify observed light-curves as a ***one-dimensional time series*** , where some ***sound property*** is is varied with ***time*** in the ***sonification***, in the same way that **flux density**, varies with ***time*** in a ***light curve*** (early in the sonification is earlier observation time, later in the sonification is later observation time).

### 3.1 Trying the `Events` source function

In `strauss` we could treat each ***flux density*** data point in the light curve as separate audio `Events`, with an occurence `time` mapped from their ***time***. 

However, articulating each data point as a separate ***note*** for ***many thousands*** of data points can require long and drawn-out sonifications. 

Here we demonstrate this approach with the light curves. We use the `Synthesizer` object with the `pitch_mapper` preset by default - this has a default pitch range of ***two octaves*** (a factor of 4 in frequency) and we pick an `E3` note (165 Hz) as the base (lowest) frequency.

We will hear the **flux** of each point as `pitch`, with their time mapped to the occurence `time` (moving from low to high time).

In [ ]:
display(Markdown(f"### Sonifying in 1D using '`Events`' object:"))

# read in the various data file - can uncomment lines to hear other files - the last
# uncommented `data_table` line will be the displayed file. 
data_table = np.genfromtxt("./AU2_Group4/Data/csv/GALEX_NUV_LC.csv", delimiter=',')[1:]
#data_table = np.genfromtxt("./AU2_Group4/Data/csv/tic_lc.csv", delimiter=',')[1:]
#data_table = np.genfromtxt("./AU2_Group4/Data/csv/kid11616200_lc.csv", delimiter=',')[1:]

# grab times and fluxes from chosen data file
time = data_table[:,0]
flux = data_table[:,1]

# show light curve again, for reference
plt.scatter(time, flux, s=4)
plt.ylabel('Flux Density')
plt.xlabel('Time')
plt.show()

%matplotlib notebook
# specify the base notes used. In this example we use a single E3 note and 
# freely vary the pitch via the 'pitch_shift' parameter 
notes = [["E3"]]

# we could also just specify a particular frequency...
#notes = [[150.]]

score =  Score(notes, 30)


data = {'pitch':np.ones(flux.size),
        'time': time,
        'pitch_shift':flux}

# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"

# set up synth (this generates the sound using mathematcial waveforms)
generator = Synthesizer()
generator.load_preset('pitch_mapper')

# or maybe the sampler instead by uncommenting this block (this uses recorded audio clips)
#generator = Sampler(sampfiles="./strauss/data/samples/mallets")
#generator.modify_preset({'phi': 0,'theta':0,})

generator.modify_preset({'note_length':0.15,
                         'volume_envelope': {'use':'on',
                                             'D':0.1,
                                             'S':0.,
                                             'A':0.01}})

# set 0 to 100 percentile limits so the full pitch range is used...
# setting 0 to 101 for pitch means the sonification is 1% longer than 
# the time needed to trigger each note - by making this more than 100%
# we give all the notes time to ring out (setting this at 100% means
# the final note is triggered at the momement the sonification ends)
lims = {'time': ('0','101'),
        'pitch_shift': ('0','100')}

# set up source
sources = Events(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions(map_lims=lims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

The articulation of each note lets you hear each data point separately, but over many noisy data points can lead to a confusing result - it's hard to keep track of the fluxes and our pitch memory is challenged.

Instead we can try ***smoothly evolving*** parameters, designating the light curves as an ***evolving `Object`*** source class. We demonstrate this in the following subsection `3.1`.

### 3.2 Instead Trying the `Objects` Source Type:

Let's first try the evolving `Object` approach without modifying the raw data in any way. In analogy to visual display, the `Event` representation is like plotting the light curve as a scatter plot, while an `Object` representation is like plotting the light curve as a continuous line.

 We set up some parameters for `strauss`, e.g. choosing a sonification length of `40` seconds. A longer sonification might let you hear more detail, but will take longer to listen to (and for `Colab` to load!)

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"

# length of the sonification in s
length = 40.

# set up synth and turn on LP filter
generator = Synthesizer()
generator.load_preset('pitch_mapper')
generator.preset_details('pitch_mapper')

Let's pick a light curve to sonify (as in Section 2). We will again default to ***Type 1*** light curve `'51788-0386-086'`.

Try changing this if you want to explore different light curves!

In [ ]:
# read in the various data file - can uncomment lines to hear other files - the last
# uncommented `data_table` line will be the displayed file. 
data_table = np.genfromtxt("./AU2_Group4/Data/csv/GALEX_NUV_LC.csv", delimiter=',')[1:]
#data_table = np.genfromtxt("./AU2_Group4/Data/csv/tic_lc.csv", delimiter=',')[1:]
#data_table = np.genfromtxt("./AU2_Group4/Data/csv/kid11616200_lc.csv", delimiter=',')[1:]

# grab times and fluxes from chosen data file
time = data_table[:,0]
flux = data_table[:,1]

# Finally, plot the light curve to remind us what we're working with
plt.plot(time,flux)
plt.xlabel(f'Time')
plt.ylabel(f'Flux')
plt.show()

let's hear the 1D time-series data sonification, mapping flux density to pitch for the light curve.

*NB: If you were wondering why `strauss` refers to the varying pitch mapping as `pitch_shift` and not just `pitch`, this is because all sources in `strauss` also need a base `pitch` which is chosen from the `'notes'` structure (here always `'A2'`) by the `Score`. This is because `strauss` can play many sources at the same time! Again, you can read more about this [in the docs](https://strauss.readthedocs.io/en/latest/).*

In [ ]:
display(Markdown(f"### Sonifying in 1D using '`pitch_shift`':"))

# show light curve again, for reference
plt.plot(time,flux)
plt.xlabel(f'Time')
plt.ylabel(f'Flux')
plt.show()

%matplotlib notebook
notes = [["A3"]]
score =  Score(notes, length)

data = {'pitch':1.,
        'time_evo':time,
        'pitch_shift':flux}

# set 0 to 100 percentile limits so the full pitch and time range is used...
lims = {'time_evo': ('0','100'),
        'pitch_shift': ('0','100')}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions(map_lims=lims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

How about mapping a low-pass filter to flux density? 

*using a 'tonal' carrier, uncomment line for a 'textural' (or white noise) carrier*

In [ ]:
display(Markdown(f"### Sonifying in 1D using low-pass '`cutoff`' frequency:"))

# show light curve again, for reference
plt.plot(time,flux)
plt.xlabel(f'Time')
plt.ylabel(f'Flux')
plt.show()


%matplotlib notebook
generator = Synthesizer()
generator.modify_preset({'filter':'on'})

# uncomment these lines to try a 'textural' sonification using white noise!
# generator.load_preset('windy')
# generator.preset_details('windy')

# we use a (power!) 'chord' here to create more harmonic richness...
notes = [["A2", "E3", 'A3', 'E4']]
score =  Score(notes, length)

data = {'pitch':[0,1,2,3],
        'time_evo':[time]*4,
        'cutoff':[flux]*4}

lims = {'time_evo': ('0','100'),
        'cutoff': ('0','100')}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
plims = {'cutoff': (0.25,0.9)}
sources.apply_mapping_functions(map_lims=lims, param_lims=plims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()

# change back in case cells are run out of order...
generator.load_preset('pitch_mapper')
%matplotlib inline

In fact, there are many expressive properties of sound we could use to represent the data in a similar way. In `strauss` these are referred to as `mappable` properties. 

A subset of these can be used as an evolving property with the `Object` source class. These are referred to as `evolvable` properties. 

Lets show what's available:

In [ ]:
display(Markdown(f"### ***'Mappable'*** properties:"))
for m in Sources.mappable:
  display(Markdown(f' * `{m}` '))

display(Markdown(f"### ***'Evolvable'*** properties:"))
for m in Sources.evolvable:
  display(Markdown(f' * `{m}` '))

We can play around with some of these `evolvable` properties here. Are all of these effective for this data? Could they be effective for other types of data representations?

The `idx` variable below controls which evolvable property is selected from the `some_mappings` list. `idx` can be changed to a number from 0 to 5 (inclusive) to *'index'* a certain sound parameter.

In [ ]:
# A list of some 'evolvable' mappings
some_mappings = ["volume", 
                 "phi",
                 "volume_lfo/amount", 
                 "volume_lfo/freq_shift",
                 "pitch_lfo/amount", 
                 "pitch_lfo/freq_shift"]

# change this (between 0 and 5) to select a different property to map...
idx = 1

# use a stereo system to allow 'phi' mapping (low pan left and high pan right)
system = "stereo"

display(Markdown(f"### Sonifying in 1D using `evolvable` property - ***`{some_mappings[idx]}`***:"))

# show light curve again, for reference
plt.plot(time,flux)
plt.xlabel(f'Time')
plt.ylabel(f'Flux')
plt.show()

%matplotlib notebook
generator = Synthesizer()
generator.modify_preset({'filter':'on',
                         "pitch_hi":-1, "pitch_lo": 1,
                         "pitch_lfo": {"use": "on", 
                                       "amount":1*("pitch_lfo/freq_shift" == some_mappings[idx]), 
                                       "freq":3*5**("pitch_lfo/freq_shift" != some_mappings[idx]), 
                                       "phase":0.25},
                         "volume_lfo": {"use": "on", 
                                        "amount":1*("volume_lfo/freq_shift" == some_mappings[idx]), 
                                        "freq":3*5**("volume_lfo/freq_shift" != some_mappings[idx]), 
                                        "phase":0}
                        })

# try a different chord (stacking fifths)...
notes = [["A2","E3","B4","F#4"]]

# chords and can also be specified via chord names and base octave....
# notes = "Em6_3"


score =  Score(notes, length)

data = {'pitch':[0,1,2,3],
        'time_evo':[time]*4,
        'cutoff':[0.9]*4,
        'theta':[0.5]*4,
        some_mappings[idx]:[(flux - flux.min())/(flux.max()-flux.min())]*4}

lims = {'time_evo': ('0','100'),
        "volume": ('0','100'), 
        "phi": (-0.5,1.5),
        "volume_lfo/amount": ('0','100'), 
        "volume_lfo/freq_shift": ('0','100'),
        "pitch_lfo/amount": ('0','100'), 
        "pitch_lfo/freq_shift": ('0','100')}

# set up source
sources = Objects(data.keys())
sources.fromdict(data)
plims = {'cutoff': (0.25,0.9)}
sources.apply_mapping_functions(map_lims=lims, param_lims=plims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

# change back in case cells are run out of order...
generator.load_preset('pitch_mapper')
system = 'mono'


### 3.X A Side Note About Presets

You can see all the parameters that make up the default `Synthesizer` preset in a pop-up window:

In [ ]:
%pycat /usr/local/lib/python3.7/dist-packages/strauss/presets/synth/default.yml

... as well as the other presets we might want to use

In [ ]:
ls -1 /usr/local/lib/python3.7/dist-packages/strauss/presets/synth/*.yml

We can modify these presets at runtime (as we do in various examples throughout this session), or even write our own presets in `.yml` format.

The `generator.preset()` function also accepts a filepath to a custom presets, where any changed preset parameters replace those in the `default` preset. 

## **4. 'Musical' example**

In previous examples, we have allowed pitch to freely vary between data points. In this example, we try ***binning the pitches*** onto musical scale intervals to make a more ***'musical'*** (in the cultural context of Western music theory) example.

To do this, we return to the `Event` based sonification, where we now use the quantised `'pitch'` parameter (as opposed to the continuous `'pitch_shift` parameter) to represent flux.

Again, you can comment and uncomment code in this cell to try some alternatives. What scale works best? What `pitch_binning` mode? Do different approaches work better for different data?

In [ ]:
display(Markdown(f"### Sonifying in 1D using '`Events`' object:"))

# read in the various data file - can uncomment lines to hear other files - the last
# uncommented `data_table` line will be the displayed file. 
data_table = np.genfromtxt("./AU2_Group4/Data/csv/GALEX_NUV_LC.csv", delimiter=',')[1:]
#data_table = np.genfromtxt("./AU2_Group4/Data/csv/tic_lc.csv", delimiter=',')[1:]
#data_table = np.genfromtxt("./AU2_Group4/Data/csv/kid11616200_lc.csv", delimiter=',')[1:]

# grab times and fluxes from chosen data file
time = data_table[:,0]
flux = data_table[:,1]

# show light curve again, for reference
plt.scatter(time, flux, s=4)
plt.ylabel('Flux Density')
plt.xlabel('Time')
plt.show()

%matplotlib notebook

# specify the notes used. 

# C major pentatonic
notes = [["C3","E3","F3","G3","B3","C4","E4","F4","G4","B4","C5","E5","F5","G5","B5"]]

# or a whole-tone scale, (was it just a dream... a dream... a dream...)
#
# This is a symetrical scale and here also demonstrates using raw frequencies in Hz.
# The whole tone scale uses 2 semitone jumps, what about minor thirds (3) fourths (5)
# or fifths (7)? Specify `semitones` to change this.
# `nint` specifies the number of intervals in the scale. Note a bigger semitone interval
# will reach higher pitches (perhaps inaudible ones...)

# semitones = 2.
# nint = 10
# notes = [100*2**(np.arange(nint)*(semitones/12))]

# ------
# In this context, we also consider the `Score` optional parameter `pitch_binning`.
#
# This determines how we bin the 'pitch' quantity, which can be either:
# - 'uniform':  the range of mapped values for 'pitch' are split into even bins
#               representing  each of the scored notes
# - 'adaptive': the range of mapped values for 'pitch' split by percentile. 
#               such that each interval is played approximately the same number of 
#               times
# without specifying,  'adaptive' is used by default, but we specify 'uniform' here
#
# which is better for representing this data?

# we specify 'uniform' pitch binning in the primary example...
score =  Score(notes, 30, pitch_binning='uniform')

# what about adaptive?
#score =  Score(notes, 30, pitch_binning='adaptive')

data = {'pitch':flux,
        'time': time}

# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"

# set up synth (this generates the sound using mathematical waveforms)
generator = Synthesizer()
generator.load_preset('pitch_mapper')

generator.modify_preset({'note_length':0.15,
                         'volume_envelope': {'use':'on',
                                             'D':0.1,
                                             'S':0.,
                                             'A':0.01,
                                             'R':0}})

# set 0 to 100 percentile limits so the full pitch range is used...
# setting 0 to 101 for pitch means the sonification is 1% longer than 
# the time needed to trigger each note - by making this more than 100%
# we give all the notes time to ring out (setting this at 100% means
# the final note is triggered at the momement the sonification ends)
lims = {'time': ('0','101'),
        'pitch_shift': ('0','100'),
        'pitch': ('0','100')}

# set up source
sources = Events(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions(map_lims=lims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

## ***5. Bonus*** 

Here, we try some bonus material if there's time, including ***spectral representations*** and ***multivariate data***.

### ***5.1. Spectral Representation***

🚧 ***under construction*** 🚧

For this type of data, a more unorthodox representation could be a ***spectral representation.***
Here, we will represent the light curves as spectra, using the `Spectralizer` generator class in `strauss` to generate a sound signal with this spectral form.

A complexity with this approach is that ***positive features*** are much more clearly represented than ***negative features*** or a ***continuum***. For this reason it makes sense to subtract some representative value (to *"zero the continuum"*) and then represent postive or negative features above it. In the example below, we can demonstrate spectralizing the light curves as-is, or for super- and sub-median brightness features only. *Do any of these effectively represent the data? Do different light curves require different approaches?*

Again, there are a number of commented code blocks in the below example that demonstrate this.

*"Spectralisation"* is covered in much more detail in the other group notebook [here](https://github.com/james-trayford/AudibleUniverseWorkbooks/blob/group3/STRAUSSdemo.ipynb)!

In [ ]:
display(Markdown(f"### Sonifying using spectral representation:"))

# read in the various data file - can uncomment lines to hear other files - the last
# uncommented `data_table` line will be the displayed file. 
data_table = np.genfromtxt("./AU2_Group4/Data/csv/GALEX_NUV_LC.csv", delimiter=',')[1:]
#data_table = np.genfromtxt("./AU2_Group4/Data/csv/tic_lc.csv", delimiter=',')[1:]
#data_table = np.genfromtxt("./AU2_Group4/Data/csv/kid11616200_lc.csv", delimiter=',')[1:]

# grab times and fluxes from chosen data file
time = data_table[:,0]
flux = data_table[:,1]

# compute the super- and sub-median features of the light curve
flux_med = np.percentile(flux, 50)
flux_hi = np.clip(flux, flux_med, np.inf) - flux_med
flux_lo = flux_med - np.clip(flux, -np.inf, np.percentile(flux, 50))

# raising the flux to a power ('contrast') of > 1 can accentuate peaks.
contrast = 1.
#contrast = 2.
#contrast = 4.

# specify the base notes used. In this example we use a single E3 note and 
# freely vary the pitch via the 'pitch_shift' parameter 
notes = [["E3"]]

score =  Score(notes, 6)

data = {'pitch': [1],
        'volume_envelope/D':[0.6], 
        'volume_envelope/S':[0.], 
        'volume_envelope/A':[0.05]}

# set the spectrum (raised by 'contrast').  
data['spectrum'] = [(flux)**contrast]
#data['spectrum'] = [(flux_hi)**contrast]
#data['spectrum'] = [(flux_lo)**contrast]


# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"

# set up synth (this generates the sound using mathematcial waveforms)
generator = Spectralizer()

# we can modify the frequency range (in Hz) like this: 
#generator.modify_preset({'min_freq':100, 'max_freq':2000})

# Illustrate with some plots (can ignore the details here)
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(121)
bdx = abs((flux-flux_med)**contrast) == abs(data['spectrum'][0])
plt.plot(time, flux, label='Light Curve')
if bdx.any():
    plt.scatter(time[bdx], flux[bdx],c='C1', label='Sonified data points')
plt.legend(frameon=0)
plt.xlabel('Time')
plt.ylabel('Flux')
ax = fig.add_subplot(122)
plt.plot(np.linspace(generator.preset['min_freq'], 
                     generator.preset['max_freq'],
                     flux.size), data['spectrum'][0], 
         label='Sonified Spectrum')
plt.legend(frameon=0)
plt.xlabel('Frequency [Hz]')
plt.show()

# set 0 to 100 percentile limits so the full pitch range is used...
# setting 0 to 101 for pitch means the sonification is 1% longer than 
lims = {'frequency': ('0','100'),
        'spectrum': ('0','100')}

# set up source
sources = Events(data.keys())
sources.fromdict(data)
sources.apply_mapping_functions(map_lims=lims)

%matplotlib notebook
soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

### ***5.2. Multivariate Sonification***

In the above examples, we have considered a single value varying with an independent variable, but can we also try sonifying multivariate data? 

***For this there is a bonus notebook located [here](https://github.com/james-trayford/AudibleUniverseWorkbooks/blob/group3/STRAUSSdemo_bonus.ipynb)***!